In [112]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import multiprocessing

In [113]:
df_2024 = pd.read_parquet('Futures_2024.parquet', engine='pyarrow')
df_2023 = pd.read_parquet('Futures_2023.parquet', engine='pyarrow')

In [114]:
df = pd.concat([df_2023, df_2024])
df = df[[col for col in df.columns if 'Close' in col]]
df.index = pd.to_datetime(df.index)
filter1 = df.dropna(axis=1, thresh=138428)
filter2 = filter1.loc[:, (df != 0).all(axis=0)]
base_df = filter2.resample('D',origin='2023-01-02 09:16:00').first().dropna()

In [115]:
df_nifty= pd.read_parquet('Idx_fut.parquet', engine='pyarrow')
df_nifty = df_nifty[[col for col in df_nifty if 'NIFTY_Close' in col]]
df_nifty.index = pd.to_datetime(df_nifty.index)
df_nifty = df_nifty[df_nifty.index > '2023-01-02 09:16:00']
nifty_frame = df_nifty.resample('D',origin='2023-01-02 09:16:00').first().dropna()
nifty_frame.drop(columns = 'BANKNIFTY_Close', inplace=True)
nifty_frame = nifty_frame[nifty_frame.index <'2024-07-02 15:30:00']

In [116]:
rel_prices = base_df.div(nifty_frame['NIFTY_Close'], axis=0)

In [117]:
nifty_50 = ["MARUTI_Close", "ITC_Close", "M&M_Close", "TITAN_Close", "EICHERMOT_Close", "HEROMOTOCO_Close", "SBIN_Close", "HINDUNILVR_Close", "TATAMOTORS_Close", "LT_Close", "ICICIBANK_Close", "BAJAJFINSV_Close", "HDFCLIFE_Close", "SBILIFE_Close", "JSWSTEEL_Close", "HINDALCO_Close", "ASIANPAINT_Close", "ADANIPORTS_Close", "HDFCBANK_Close", "COALINDIA_Close", "TCS_Close", "INFY_Close", "NTPC_Close", "AXISBANK_Close", "APOLLOHOSP_Close", "TATASTEEL_Close", "BHARTIARTL_Close", "ADANIENT_Close", "INDUSINDBK_Close", "TECHM_Close", "KOTAKBANK_Close", "BAJFINANCE_Close", "BPCL_Close", "SHRIRAMFIN_Close", "RELIANCE_Close", "ONGC_Close"]
stock_prices = base_df[nifty_50]
relative_prices = rel_prices[nifty_50]
full_price_data = filter2[nifty_50]

In [118]:
adjusted_index = full_price_data.index - pd.Timedelta(seconds=59)
full_price_data.index = adjusted_index

In [119]:
multiplier = pd.read_csv('instrument_list.csv')
multiplier = multiplier.drop(columns=[col for col in multiplier.columns if col not in ['name', 'segment', 'lot_size']])
multiplier_nfo = multiplier[multiplier['segment'] == 'NFO-FUT']
filtered_df = multiplier_nfo.drop_duplicates(subset='name', keep='first')
valid_stocks = filtered_df['name'].unique()
valid_stock_columns = [f"{stock}_Close" for stock in valid_stocks]
columns_to_keep = [col for col in valid_stock_columns if col in stock_prices.columns]
filtered_price_df = stock_prices[columns_to_keep]
filtered_stock_names = [col.split('_')[0] for col in filtered_price_df.columns]
filtered_multiplier_df = filtered_df[filtered_df['name'].isin(filtered_stock_names)]
filtered_multiplier_df.drop(columns = 'segment', inplace=True)
filtered_multiplier_df.set_index('name', inplace=True)
filtered_multiplier_df.columns = ['mult']
mult_list = filtered_multiplier_df
mult_list.index = mult_list.index + '_Close'

C:\Users\sambu\AppData\Local\Temp\ipykernel_5044\984550555.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_multiplier_df.drop(columns = 'segment', inplace=True)


In [120]:
class PortfolioMomentumStrategy:
    def __init__(self, stock_prices, relative_prices, full_price_data, mult, lookback_period, num_stocks, max_positions, initial_capital=200000000):
        self.stock_prices = stock_prices
        self.relative_prices = relative_prices
        self.full_price_data = full_price_data
        self.mult = mult
        self.max_positions = max_positions
        self.lookback_period = lookback_period
        self.num_stocks = int(min(max_positions // 2, num_stocks))
        self.available_capital = initial_capital
        self.current_positions = {stock: 0 for stock in stock_prices.columns}
        self.pnl_series = pd.Series(index = stock_prices.index)
        self.trade_log = []
        self.exit_log = []
        self.transaction_costs_rate = 0.0011
        self.position_matrix = pd.DataFrame(index=stock_prices.index, columns=stock_prices.columns)
        #self.position_df = pd.DataFrame(index = full_price_data, columns = stock_prices.columns)
        

    def calculate_momentum(self, data, current_time):
        lookback_time = current_time - pd.Timedelta(days=self.lookback_period)
        if lookback_time in data.index:
            return (data.loc[current_time]/ data.loc[lookback_time] - 1)  # Returns the momentum between now and that time
        else:
            return pd.Series(np.nan, index=data.columns) #In cases of large lookback period
    
    def rank_stocks(self, current_time):
        momentum_scores = self.calculate_momentum(self.relative_prices, current_time)
        ranked_stocks = momentum_scores.sort_values(ascending = False) #Ranks stocks top to bottom of momentum in the time period
        return ranked_stocks
    
    def allocate_positions(self, ranked_stocks, current_prices):
        top_stocks = ranked_stocks.head(self.num_stocks)
        bottom_stocks = ranked_stocks.tail(self.num_stocks)


        long_value = self.available_capital / 2
        short_value = self.available_capital / 2 #Assign half capital to long, half to short

        max_exp_per_stock = 10000000

        long_positions = {}  #To track which stocks we go long and short on and the position size
        short_positions = {}

        #So we don't repeatedly buy/sell if a stock remains in the top/bottom n stocks
        top_stocks = [stock for stock in top_stocks.index if self.current_positions[stock] == 0]
        bottom_stocks = [stock for stock in bottom_stocks.index if self.current_positions[stock] == 0]

        #Tracks total positions, i.e number of stocks in our portfolio
        total_positions = sum(1 for pos in self.current_positions.values() if pos != 0)

        #Go long with the top n
        for stock in top_stocks:
            if total_positions < self.max_positions:  #Only if we don't go over max_positions stocks
                
                #Position size will be the minimum of the exposure per stock/ (total price of contract) and available capital / (total price of contract)
                
                position_size = int(min(max_exp_per_stock / (current_prices[stock] * self.mult.loc[stock].values), long_value / (current_prices[stock] * self.mult.loc[stock].values)))
                position_cost = position_size * current_prices[stock] * self.mult.loc[stock].values
                
                #Only carry out trade if we have enough capital
                
                if position_cost <= long_value:
                    long_positions[stock] = position_size
                    long_value -= position_cost
                    self.trade_log.append({'date': current_prices.name, 'stock': stock, 'position': 'long', 'size': position_size, 'cost': position_cost[0], 'trading price' : current_prices[stock], 'trade_type' : 'entry', 'multiplier': self.mult.loc[stock].values[0]})
                    total_positions += 1

        #Same process, but for short.
        for stock in bottom_stocks:
            if total_positions < self.max_positions:
                position_size = int(min(max_exp_per_stock / (current_prices[stock] * self.mult.loc[stock].values), short_value / (current_prices[stock] * self.mult.loc[stock].values)))
                position_cost = position_size * current_prices[stock] * self.mult.loc[stock].values
                if position_cost <= short_value:
                    short_positions[stock] = position_size
                    short_value -= position_cost
                    self.trade_log.append({'date': current_prices.name, 'stock': stock, 'position': 'short', 'size': position_size, 'cost': position_cost[0], 'trading price': current_prices[stock], 'trade_type' : 'entry', 'multiplier': self.mult.loc[stock].values[0]})
                    total_positions += 1

        return long_positions, short_positions  


    def update_positions(self, long_positions, short_positions, previous_prices, current_prices):

        liquidated_pnl = 0
        total_positions = sum(1 for pos in self.current_positions.values() if pos != 0)

        #If stocks move out of the top and bottom n, the position is closed and we calculate the pnl of the close

        for stock in self.current_positions:
            if stock not in long_positions and stock not in short_positions:
                if self.current_positions[stock] != 0:
                    liquidated_pnl = self.current_positions[stock] * (current_prices[stock]) * self.mult.loc[stock].values
                    total_positions -= 1
                    self.exit_log.append({'date': current_prices.name, 'stock': stock, 'position': ' Closing long' if self.current_positions[stock] > 0 else ' Closing short',
                                        'size': abs(self.current_positions[stock]),
                                        'cost': abs(self.current_positions[stock]) * previous_prices[stock] * self.mult.loc[stock].values[0],
                                        'trading price': current_prices[stock],
                                        'trade_type': 'exit', 'multiplier': self.mult.loc[stock].values[0]})
                    self.current_positions[stock] = 0

        #Current_positions tracks the stocks in our portfolio and the position size
        
        for stock, position in long_positions.items():
            self.current_positions[stock] = position

        for stock, position in short_positions.items():
            self.current_positions[stock] = -position

        return liquidated_pnl
    
    def update_position_matrix(self, current_time):
        current_prices = self.stock_prices.loc[current_time]
        for stock, position in self.current_positions.items():
            self.position_matrix.at[current_time, stock] = position
  
    #Simple function to track pnl

    def calculate_pnl(self, long_positions, short_positions, previous_prices, current_prices):
        pnl = 0

        for stock, position in self.current_positions.items():
            price_change = current_prices[stock] - previous_prices[stock]
            pnl += position * price_change * self.mult.loc[stock].values + self.update_positions(long_positions, short_positions, previous_prices, current_prices)
        return pnl
    
    # def calculate_minutely_pnl(self, current_time):
    #     pnl = 0
    #     for stock, position in self.current_positions.items():
    #         price_change = self.full_price_data.loc[current_time] - self.full_price_data.shift(1).loc[current_time]
    #         pnl += position * price_change * self.mult.loc[stock].values
    #     return pnl
    
    
    def calculate_transaction_costs(self, trades):
        total_transaction_costs = 0
        for trade in trades:
            turnover = trade['size'] * trade['trading price']
            total_transaction_costs += turnover * self.transaction_costs_rate
        return total_transaction_costs

    def trade(self):

        #Indexing to be able to select correct prices

        unique_dates=self.stock_prices.index
        unique_dates=unique_dates[self.lookback_period:]
        # start_timestamp = unique_dates[0]

        #unique_minute_dates = self.full_price_data.loc[start_timestamp:].index
        
        next_unique_dates=self.stock_prices.index
        next_unique_dates=next_unique_dates[self.lookback_period-1:]

        #For every day, apply the strategy
        for current_time,prev_time in zip(unique_dates,next_unique_dates):
            previous_prices = self.stock_prices.loc[prev_time]
            current_prices = self.stock_prices.loc[current_time]
            ranked_stocks = self.rank_stocks(current_time)
            long_positions, short_positions = self.allocate_positions(ranked_stocks, current_prices)
            self.update_positions(long_positions, short_positions, previous_prices, current_prices)
            pnl_raw = self.calculate_pnl(long_positions, short_positions, previous_prices, current_prices) 
            liquidated_pnl = self.update_positions(long_positions, short_positions, previous_prices, current_prices)
            pnl = pnl_raw + liquidated_pnl
            self.update_position_matrix(current_time)

            day_trades = [trade for trade in self.trade_log if trade['date'] == current_time]
            if day_trades:  
                transaction_costs = self.calculate_transaction_costs(day_trades)
                pnl -= transaction_costs
            
            self.pnl_series[current_time] = pnl 

            self.available_capital += pnl

        return pd.Series(self.pnl_series, index=self.stock_prices.index[self.lookback_period:]).cumsum(), self.position_matrix
    
    


    # def calculate_minutely_pnl(self):
    #     minutely_pnl_series = pd.Series(index=self.full_price_data.index, dtype=float)
    #     previous_prices = self.full_price_data.shift(1)

    #     pnl_changes = self.full_price_data.sub(previous_prices).mul(pd.Series(self.current_positions) * self.mult.values.flatten(), axis=1).sum(axis=1)
    #     minutely_pnl_series.iloc[:] = pnl_changes.values


    #     minutely_pnl_series = minutely_pnl_series.cumsum()
    #     return minutely_pnl_series
    



In [121]:
strategy = PortfolioMomentumStrategy(stock_prices, relative_prices, full_price_data, mult_list, 10, 5, 20)
pnl_series, position_matrix = strategy.trade()

C:\Users\sambu\AppData\Local\Temp\ipykernel_5044\1367176806.py:58: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  position_size = int(min(max_exp_per_stock / (current_prices[stock] * self.mult.loc[stock].values), long_value / (current_prices[stock] * self.mult.loc[stock].values)))
C:\Users\sambu\AppData\Local\Temp\ipykernel_5044\1367176806.py:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  position_size = int(min(max_exp_per_stock / (current_prices[stock] * self.mult.loc[stock].values), short_value / (current_prices[stock] * self.mult.loc[stock].values)))


In [122]:
pnl_series

DateTime
2023-01-12 09:16:00    3.102763e+05
2023-01-13 09:16:00    7.115695e+05
2023-01-15 09:16:00    8.597625e+05
2023-01-16 09:16:00    8.848559e+05
2023-01-17 09:16:00    7.091648e+05
                           ...     
2024-06-27 09:16:00    1.637721e+08
2024-06-28 09:16:00    1.634449e+08
2024-06-30 09:16:00    1.636499e+08
2024-07-01 09:16:00    1.636716e+08
2024-07-02 09:16:00    1.635337e+08
Length: 449, dtype: float64

In [123]:
position_matrix.dropna(inplace=True)

In [124]:
full_price_data = full_price_data[full_price_data.index >= '2023-01-16 09:14:00']

##### Calculate minutely price change

In [125]:
price_change = full_price_data.diff()

In [126]:
#position_matrix

In [127]:
stock_prices_daily = stock_prices[stock_prices.index >= '2023-01-16 09:15:00']

In [128]:
stock_prices_daily.to_csv('daily_stock_prices.csv')

In [129]:
position_matrix.to_csv('daily_position_matrix.csv')

In [130]:
minutely_position_matrix = pd.DataFrame(index = full_price_data.index, columns = position_matrix.columns)

In [131]:
for column in position_matrix.columns:
    minutely_position_matrix[column] = minutely_position_matrix[column].fillna(method='ffill')

minutely_position_matrix.update(position_matrix.reindex(minutely_position_matrix.index, method='ffill'))

C:\Users\sambu\AppData\Local\Temp\ipykernel_5044\1435253019.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  minutely_position_matrix[column] = minutely_position_matrix[column].fillna(method='ffill')
C:\Users\sambu\AppData\Local\Temp\ipykernel_5044\1435253019.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  minutely_position_matrix[column] = minutely_position_matrix[column].fillna(method='ffill')


In [132]:
full_price_data.to_csv('full_price_data.csv')

In [133]:
mult_list.to_csv('mult_list.csv')

In [134]:
minutely_position_matrix.to_csv('minutely_position_matrix.csv')


In [135]:
minutely_price_position = minutely_position_matrix.mul(price_change)

In [136]:
minutely_price_position.dropna(inplace=True)

In [137]:
mult_df = mult_list.T
mult_df = mult_df.reindex(columns=minutely_price_position.columns)

minutely_stock_pnl = minutely_price_position * mult_df.values


In [138]:
#minutely_stock_pnl

In [139]:
minutely_pnl = minutely_stock_pnl.sum(axis=1)

In [140]:
minutely_pnl.to_csv('minutely_pnl.csv')

In [141]:
initial_cap = 200000000

In [142]:
cumulative_minutely_pnl = minutely_pnl.cumsum()

In [143]:
rel_cumulative_minutely_pnl = cumulative_minutely_pnl / initial_cap

In [144]:
rel_cumulative_minutely_pnl.to_csv('minute_pnl_script1.csv')

In [145]:
# initial_cap = 200000000
# relative_series = pnl_series/initial_cap
# relative_series

In [146]:
trade_log_df = pd.DataFrame(strategy.trade_log)
exit_log_df = pd.DataFrame(strategy.exit_log)
full_trade_log = pd.concat([trade_log_df, exit_log_df], ignore_index=True)

full_trade_log.sort_values(by=['date', 'trade_type'], ascending=[True, False], inplace=True)


full_trade_log.reset_index(drop=True, inplace=True)

In [147]:
#full_trade_log

In [148]:
initial_cap = 200000000
pct = 100
daily_ret = pct* (cumulative_minutely_pnl.diff() / initial_cap)
annual_mean_returns = daily_ret.mean()*252*1440
annial_mean_std = daily_ret.std()*np.sqrt(252*1440)
s_r = annual_mean_returns/ annial_mean_std
print('Annual Mean Return:', annual_mean_returns,
       'Annual_Risk:', annial_mean_std,
        'Sharpe:', s_r)

Annual Mean Return: 25.036663984495185 Annual_Risk: 8.801100999993496 Sharpe: 2.8447195395796148
